In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
train_err  = pd.read_csv('LG/err_group_train_target.csv')
train_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,NaN
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,2.020111e+13
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,2.020111e+13
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,NaN
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,2.020112e+13
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,2.020112e+13
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,2.020111e+13
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,2.020111e+13


In [3]:
train_err['time']=train_err['time'].fillna(0)

In [4]:
train_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.000000e+00
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,2.020111e+13
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,2.020111e+13
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.000000e+00
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,2.020112e+13
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,2.020112e+13
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,2.020111e+13
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,2.020111e+13


In [5]:
train_err_=train_err.copy()
for x in range(24846):
    if train_err_['time_min'][x]<train_err_['time'][x]<train_err_['time_max'][x]:
        train_err_['time'][x]=1
    else:
        train_err_['time'][x]=0

In [6]:
train_err_

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.0
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,0.0
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,1.0
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,1.0
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,0.0
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,0.0
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,1.0


In [9]:
T=train_err_.groupby(['user_id','fwver'])['time'].sum()

In [12]:
T

user_id  fwver     
10000    05.15.2138    0.0
10001    04.33.1185    0.0
         04.33.1261    1.0
10002    05.15.2138    0.0
10003    04.33.1185    0.0
                      ... 
24997    04.22.1750    1.0
         04.22.1778    0.0
24998    04.22.1750    0.0
         04.22.1778    1.0
24999    05.15.2138    0.0
Name: time, Length: 24062, dtype: float64

In [14]:
T1=list(T)
for x in range(24062):
    if T1[x]>0:
        T1[x]=1

In [30]:
T2=pd.DataFrame(T1)
T2[0].value_counts()

0.0    19293
1.0     4769
Name: 0, dtype: int64

In [31]:
train_err_dup=train_err_.drop_duplicates(['user_id','model_nm','fwver'])

In [32]:
train_err_dup

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.0
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,0.0
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,1.0
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,1.0
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,0.0
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,0.0
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,1.0


In [35]:
T2

,0
0,0.0
1,0.0
2,1.0
3,0.0
4,0.0
...,...
24057,1.0
24058,0.0
24059,0.0
24060,1.0


In [36]:
train_err_dup["time"]=list(T2[0])
train_err_dup

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.0
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,0.0
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,1.0
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,1.0
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,0.0
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,0.0
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,1.0


In [51]:
train_err_dup.index=np.arange(24062)
train_err_dup

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.0
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,0.0
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,1.0
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24057,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,1.0
24058,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,0.0
24059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,0.0
24060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,1.0


In [37]:
test_err= pd.read_csv('LG/err_group_test.csv')
test_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,20201101030227,20201127024946,30000,model_1,04.16.3553
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,20201118073855,20201122050431,30000,model_1,04.16.3571
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,20201127094708,20201130192105,30000,model_2,04.33.1261
3,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20201101042135,20201130221906,30001,model_3,05.15.2138
4,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,20201101003616,20201126040300,30002,model_0,04.22.1750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,156.0,1.0,0.0,20201101003242,20201119052538,44996,model_0,04.22.1750
24092,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,20201119053129,20201130220212,44996,model_0,04.22.1778
24093,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,20201101000032,20201127035735,44997,model_0,04.22.1750
24094,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,5.0,0.0,20201127040356,20201130232034,44997,model_0,04.22.1778


In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #시각화 패키지
import seaborn as sns #시각화 패키지
plt.style.use('fivethirtyeight') # 테마바꾸기
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from tqdm import tnrange, tqdm_notebook, notebook
import time

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, scale

from sklearn.linear_model import RidgeCV, Ridge, LassoCV, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
import lightgbm as lgb

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans

# 실루엣 분석 metric 값을 구하기 위한 API 추가
from sklearn.metrics import silhouette_samples, silhouette_score

from sklearn.datasets import make_blobs, load_boston
import matplotlib.cm as cm

In [77]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = train_err_dup.drop(["time","time_min","time_max","user_id","fwver"],axis=1)
train_y = train_err_dup.time

In [78]:
test_x = test_err.drop(["time_min","time_max","user_id","fwver"],axis=1)

In [79]:
from sklearn.preprocessing import LabelEncoder

encoder_model_tr=LabelEncoder()
encoder_model_tr.fit(train_x['model_nm'])
label_model_tr=encoder_model_tr.transform(train_x['model_nm'])
train_x['model_nm']=label_model_tr

encoder_model_tr=LabelEncoder()
encoder_model_tr.fit(test_x['model_nm'])
label_model_tr=encoder_model_tr.transform(test_x['model_nm'])
test_x['model_nm']=label_model_tr

In [81]:
train_x.shape

(24062, 42)

In [66]:
test_x.shape

(24096, 43)

In [137]:
#run traning
LGBM=LGBMClassifier(boosting_type='gbdt',objective='binary',random_state=0)
#result=cross_val_score(LGBM,train_features, train_target,cv=10,scoring='roc_auc')
#print('The cross validated score for LightBoost is:',result.mean())

In [138]:
LGBM.fit(train_x, train_y)

LGBMClassifier(objective='binary', random_state=0)

In [139]:
proba=pd.DataFrame(test_err['user_id'])
proba['problem']=LGBM.predict_proba(test_x)[:,1]
#LGBM=LGBM.drop([0,1],axis=1)
#LGBM.to_csv(r"/content/LGBM.csv", index=False)

In [140]:
proba

,user_id,problem
0,30000,0.271975
1,30000,0.150477
2,30000,0.142310
3,30001,0.210725
4,30002,0.227982
...,...,...
24091,44996,0.579051
24092,44996,0.215678
24093,44997,0.867647
24094,44997,0.092313


In [141]:
last=proba.groupby(['user_id']).max()

In [142]:
last

,problem
user_id,
30000,0.271975
30001,0.210725
30002,0.227982
30003,0.681499
30004,0.456810
...,...
44994,0.211354
44995,0.261139
44996,0.579051


In [136]:
last.to_csv(r"LG_LGBM.csv", index=True)

In [143]:
test_err.groupby(['user_id']).mean()

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max
user_id,,,,,,,,,,,,,,,,,,,,,
30000,0.0,0.0,0.0,0.000000,20.666667,0.333333,0.333333,0.0,0.0,0.0,...,0.333333,1.000000,1.000000,0.000000,0.666667,30.666667,0.0,0.666667,2.020112e+13,2.020113e+13
30001,0.0,0.0,3.0,98.000000,3.000000,4.000000,4.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.020110e+13,2.020113e+13
30002,0.0,0.0,0.0,0.000000,39.500000,1.000000,0.500000,0.0,0.0,0.0,...,0.000000,0.500000,0.500000,0.000000,7.000000,56.500000,0.5,1.000000,2.020111e+13,2.020113e+13
30003,0.0,0.0,0.0,0.000000,53.500000,1.000000,1.000000,0.0,0.0,0.0,...,1.500000,0.500000,0.500000,0.000000,0.000000,20.000000,4.0,0.000000,2.020111e+13,2.020113e+13
30004,0.0,0.0,0.0,0.666667,2.000000,0.333333,0.333333,0.0,0.0,0.0,...,0.333333,0.333333,0.333333,0.333333,0.000000,16.000000,30.0,0.000000,2.020112e+13,2.020113e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,0.0,0.0,0.0,2.000000,12.000000,5.000000,5.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,87.0,0.000000,2.020110e+13,2.020113e+13
44995,0.0,0.0,0.0,0.000000,0.000000,1.000000,1.000000,0.0,0.0,0.0,...,0.000000,0.500000,0.500000,0.000000,0.000000,6.000000,8.5,0.000000,2.020110e+13,2.020112e+13
44996,0.0,0.0,0.0,0.000000,0.000000,0.500000,0.500000,0.0,0.0,0.0,...,0.000000,0.500000,0.500000,0.500000,0.000000,164.500000,1.0,1.500000,2.020111e+13,2.020112e+13


In [145]:
test_err.user_id.unique().shape

(14998,)